In [2]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
from code_map import meters, markets
import numpy as np
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime
import pytz
import openpyxl
from itertools import combinations, product


In [3]:
H = pd.date_range(start=pd.Timestamp(year= markets.year, month= markets.start_month, day = markets.start_day, hour = markets.start_hour), 
                              end= pd.Timestamp(year = markets.year, month = markets.end_month, day = markets.end_day, hour = markets.end_hour), freq="H", tz = "Europe/Oslo")
power_meter_dict = meters.power_meters

# Define the sets
L = list(power_meter_dict.values())  # List of PowerMeter objects
M = markets.all_market_list  # List of ReserveMarket objects

In [5]:
# Create a new model
test_model = Model("AssetToMarket")

# Create decision variables
x = {}
for h, hour in enumerate(H):
    for l, load in enumerate(L):
        for m, market in enumerate(M):
            # asset i is connected to market j at hour h
            x[h, l, m] = test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"x_{h}_{l}_{m}")
            
            # If asset direction is "up"
            if load.direction == "up" and market.direction == "down":
                test_model.addConstr(x[h, l, m] == 0, f"direction_constraint_hour_{hour}_asset_{load}_market_{market}")

            # If asset direction is "down"
            elif load.direction == "down" and market.direction == "up":
                test_model.addConstr(x[h, l, m] == 0, f"direction_constraint_hour_{hour}_asset_{load}_market_{market}")

            # If market direction is "both"
            elif market.direction == "both" and load.direction != "both":
                test_model.addConstr(x[h, l, m] == 0, f"direction_constraint_hour_{hour}_asset_{load}_market_{market}")

            # If asset direction is "any", we don't need a constraint as it can connect to any market.

y = {}
for h, hour in enumerate(H):
    for m, market in enumerate(M):
        # market j is active at hour h
        y[h, m] = test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"y_{h}_{m}")


# Set objective
test_model.setObjective(
    # maxmize income 
    sum(x[h, l, m] * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] * market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for l, load in enumerate(L) for m, market in enumerate(M) for h, hour in enumerate(H)),
    GRB.MAXIMIZE
)

# Add constraints
for h, hour in enumerate(H):
    for l, load in enumerate(L):
        # Each asset can only be connected to one market at a time
        test_model.addConstr(sum(x[h, l, m] for m in range(len(M))) <= 1, f"single_market_for_asset_at_hour_{h}_nr.{l}")

for h, hour in enumerate(H):
    for m, market in enumerate(M):
        # Connect the binary variables by using big M
        test_model.addConstr((sum(x[h, l, m] for l in range(len(L)))) <= len(L) * y[h, m], f"asset_connection_for_hour_{h}_market_{m}")
        
        # Min volume constraint
        test_model.addConstr(sum(x[h, l, m] * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L)) >= market.min_volume * y[h, m], f"min_volume_for_hour_{h}_market_{m}")

        # Weighted average constraint for response time
        total_response_time = sum(x[h, l, m] * load.response_time for l, load in enumerate(L))
        total_assigned_assets = sum(x[h, l, m] for l in range(len(L)))
        test_model.addConstr(total_response_time <= market.response_time * total_assigned_assets, f"avg_response_time_for_hour_{h}_market_{m}")

        # Max volume constraint
        test_model.addConstr(sum(x[h, l, m] * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L)) <=  market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")

# Solve the model
test_model.optimize()
    
if test_model.status == GRB.Status.INFEASIBLE:
    test_model.computeIIS()


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-18


KeyboardInterrupt: 

In [ ]:
# Retrieve and print solution (optional)
if test_model.status == GRB.Status.OPTIMAL:
    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            for m, market in enumerate(M):
                if x[h, l, m].X > 0.5:
                    print(f"Asset with meter id: {load.meter_id} is bid into {market.name} in hour {hour}")
else:
    print("Optimization was not successful!")

In [ ]:
data = []

for h, hour in enumerate(H):
    for l, load in enumerate(L):
        for m, market in enumerate(M):
            if x[h, l, m].X > 0.5:
                data.append([hour, load.meter_id, market.name])

df = pd.DataFrame(data, columns=["Hour", "Asset Meter ID", "Market"])
#print(df)


market_count_dict = {}
for hour in H:
    hour_df = df.loc[(df["Hour"] == hour)]
    # get the number of assets in each market
    market_count = hour_df.groupby(["Market", "Hour"]).agg("count").reset_index().rename(columns={"Asset Meter ID": "Asset Count"})
    market_count_dict[hour] = market_count
    

In [ ]:
for hour in H:
    display(market_count_dict[hour])